In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex SDK: AutoML image classification model


## Installation

Install the latest (preview) version of Vertex SDK.


In [ ]:
! pip3 install -U google-cloud-aiplatform --user

Install the Google *cloud-storage* library as well.


In [ ]:
! pip3 install google-cloud-storage

### Restart the Kernel

Once you've installed the Vertex SDK and Google *cloud-storage*, you need to restart the notebook kernel so it can find the packages.


In [ ]:
import os

if not os.getenv("AUTORUN") and False:
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU run-time

*Make sure you're running this notebook in a GPU runtime if you have that option. In Colab, select* **Runtime > Change Runtime Type > GPU**

### Set up your GCP project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a GCP project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the Vertex APIs and Compute Engine APIs.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component)

4. [Google Cloud SDK](https://cloud.google.com/sdk) is already installed in Google Cloud Notebooks.

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$` into these commands.


In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

In [ ]:
if PROJECT_ID == "" or PROJECT_ID is None or PROJECT_ID == "[your-project-id]":
    # Get your GCP project id from gcloud
    shell_output = !gcloud config list --format 'value(core.project)' 2>/dev/null
    PROJECT_ID = shell_output[0]
    print("Project ID:", PROJECT_ID)

In [ ]:
! gcloud config set project $PROJECT_ID

#### Region

You can also change the `REGION` variable, which is used for operations
throughout the rest of this notebook.  Below are regions supported for Vertex AI. We recommend when possible, to choose the region closest to you.

- Americas: `us-central1`
- Europe: `europe-west4`
- Asia Pacific: `asia-east1`

You cannot use a Multi-Regional Storage bucket for training with Vertex. Not all regions provide support for all Vertex services. For the latest support per region, see [Region support for Vertex AI services](https://cloud.google.com/vertex-ai/docs/general/locations)


In [ ]:
REGION = "us-central1"  # @param {type: "string"}

#### Timestamp

If you are in a live tutorial session, you might be using a shared test account or project. To avoid name collisions between users on resources created, you create a timestamp for each instance session, and append onto the name of resources which will be created in this tutorial.


In [ ]:
from datetime import datetime

TIMESTAMP = datetime.now().strftime("%Y%m%d%H%M%S")

### Authenticate your GCP account

**If you are using Google Cloud Notebooks**, your environment is already
authenticated. Skip this step.

*Note: If you are on an Vertex notebook and run the cell, the cell knows to skip executing the authentication steps.*


In [ ]:
import os
import sys

# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your Google Cloud account. This provides access
# to your Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

# If on Vertex, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this tutorial in a notebook locally, replace the string
    # below with the path to your service account key and run this cell to
    # authenticate your Google Cloud account.
    else:
        %env GOOGLE_APPLICATION_CREDENTIALS your_path_to_credentials.json

    # Log in to your account on Google Cloud
    ! gcloud auth login

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

This tutorial is designed to use training data that is in a public Cloud Storage bucket and a local Cloud Storage bucket for your batch predictions. You may alternatively use your own training data that you have stored in a local Cloud Storage bucket.

Set the name of your Cloud Storage bucket below. It must be unique across all Cloud Storage buckets.


In [ ]:
BUCKET_NAME = "[your-bucket-name]"  # @param {type:"string"}

In [ ]:
if BUCKET_NAME == "" or BUCKET_NAME is None or BUCKET_NAME == "[your-bucket-name]":
    BUCKET_NAME = PROJECT_ID + "aip-" + TIMESTAMP

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.


In [ ]:
! gsutil mb -l $REGION gs://$BUCKET_NAME

Finally, validate access to your Cloud Storage bucket by examining its contents:


In [ ]:
! gsutil ls -al gs://$BUCKET_NAME

### Set up variables

Next, set up some variables used throughout the tutorial.
### Import libraries and define constants


#### Import Vertex SDK

Import the Vertex SDK into our Python environment.


In [ ]:
import os
import sys
import time

from google.cloud.aiplatform import gapic as aip
from google.protobuf import json_format
from google.protobuf.json_format import MessageToJson, ParseDict
from google.protobuf.struct_pb2 import Struct, Value

#### Vertex AI constants

Setup up the following constants for Vertex AI:

- `API_ENDPOINT`: The Vertex AI API service endpoint for dataset, model, job, pipeline and endpoint services.
- `API_PREDICT_ENDPOINT`: The Vertex AI API service endpoint for prediction.
- `PARENT`: The Vertex AI location root path for dataset, model and endpoint resources.


In [ ]:
# API Endpoint
API_ENDPOINT = "{}-aiplatform.googleapis.com".format(REGION)

# Vertex AI location root path for your dataset, model and endpoint resources
PARENT = "projects/" + PROJECT_ID + "/locations/" + REGION

#### AutoML constants

Next, setup constants unique to AutoML image classification datasets and training:

- Dataset Schemas: Tells the managed dataset service which type of dataset it is.
- Data Labeling (Annotations) Schemas: Tells the managed dataset service how the data is labeled (annotated).
- Dataset Training Schemas: Tells the Vertex AI Pipelines service the task (e.g., classification) to train the model for.


In [ ]:
# Image Dataset type
IMAGE_SCHEMA = "google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
# Image Labeling type
IMPORT_SCHEMA_IMAGE_CLASSIFICATION = "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml"
# Image Training task
TRAINING_IMAGE_CLASSIFICATION_SCHEMA = "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml"

## Clients

The Vertex SDK works as a client/server model. On your side (the Python script) you will create a client that sends requests and receives responses from the server (Vertex).

You will use several clients in this tutorial, so set them all up upfront.

- Dataset Service for managed datasets.
- Model Service for managed models.
- Pipeline Service for training.
- Endpoint Service for deployment.
- Job Service for batch jobs and custom training.
- Prediction Service for serving. *Note*: Prediction has a different service endpoint.


In [ ]:
# client options same for all services
client_options = {"api_endpoint": API_ENDPOINT}


def create_dataset_client():
    client = aip.DatasetServiceClient(client_options=client_options)
    return client


def create_model_client():
    client = aip.ModelServiceClient(client_options=client_options)
    return client


def create_pipeline_client():
    client = aip.PipelineServiceClient(client_options=client_options)
    return client


def create_endpoint_client():
    client = aip.EndpointServiceClient(client_options=client_options)
    return client


def create_prediction_client():
    client = aip.PredictionServiceClient(client_options=client_options)
    return client


def create_job_client():
    client = aip.JobServiceClient(client_options=client_options)
    return client


clients = {}
clients["dataset"] = create_dataset_client()
clients["model"] = create_model_client()
clients["pipeline"] = create_pipeline_client()
clients["endpoint"] = create_endpoint_client()
clients["prediction"] = create_prediction_client()
clients["job"] = create_job_client()

for client in clients.items():
    print(client)

In [ ]:
IMPORT_FILE = (
    "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
)

In [ ]:
! gsutil cat $IMPORT_FILE | head -n 10

*Example output*:
```
gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10140303196_b88d3d6cec.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10172379554_b296050f82_n.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10172567486_2748826a8b.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10172636503_21bededa75_n.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/102841525_bd6628ae3c.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/1031799732_e7f4008c03.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10391248763_1d16681106_n.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10437754174_22ec990b77_m.jpg,daisy
gs://cloud-ml-data/img/flower_photos/daisy/10437770546_8bb6f7bdd3_m.jpg,daisy
```


## Create a dataset


### [projects.locations.datasets.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/create)


#### Request


In [ ]:
DATA_SCHEMA = IMAGE_SCHEMA

dataset = {
    "display_name": "flowers_" + TIMESTAMP,
    "metadata_schema_uri": "gs://" + DATA_SCHEMA,
}

print(
    MessageToJson(
        aip.CreateDatasetRequest(
            parent=PARENT,
            dataset=dataset,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "dataset": {
    "displayName": "flowers_20210226014942",
    "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml"
  }
}
```


#### Call


In [ ]:
request = clients["dataset"].create_dataset(
    parent=PARENT,
    dataset=dataset,
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/datasets/3094342379910463488",
  "displayName": "flowers_20210226014942",
  "metadataSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/metadata/image_1.0.0.yaml",
  "labels": {
    "aiplatform.googleapis.com/dataset_metadata_schema": "IMAGE"
  },
  "metadata": {
    "dataItemSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/dataitem/image_1.0.0.yaml"
  }
}
```


In [ ]:
# The full unique ID for the dataset
dataset_id = result.name
# The short numeric ID for the dataset
dataset_short_id = dataset_id.split("/")[-1]

print(dataset_id)

### [projects.locations.datasets.import](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.datasets/import)


#### Request


In [ ]:
LABEL_SCHEMA = IMPORT_SCHEMA_IMAGE_CLASSIFICATION

import_config = {
    "gcs_source": {
        "uris": [IMPORT_FILE],
    },
    "import_schema_uri": LABEL_SCHEMA,
}

print(
    MessageToJson(
        aip.ImportDataRequest(
            name=dataset_short_id,
            import_configs=[import_config],
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "3094342379910463488",
  "importConfigs": [
    {
      "gcsSource": {
        "uris": [
          "gs://cloud-samples-data/vision/automl_classification/flowers/all_data_v2.csv"
        ]
      },
      "importSchemaUri": "gs://google-cloud-aiplatform/schema/dataset/ioformat/image_classification_single_label_io_format_1.0.0.yaml"
    }
  ]
}
```


#### Call


In [ ]:
request = clients["dataset"].import_data(
    name=dataset_id,
    import_configs=[import_config],
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


## Train a model


### [projects.locations.trainingPipelines.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)


#### Request


In [ ]:
TRAINING_SCHEMA = TRAINING_IMAGE_CLASSIFICATION_SCHEMA

task = Value(
    struct_value=Struct(
        fields={
            "multi_label": Value(bool_value=False),
            "model_type": Value(string_value="CLOUD"),
            "budget_milli_node_hours": Value(number_value=8000),
            "disable_early_stopping": Value(bool_value=False),
        }
    )
)

training_pipeline = {
    "display_name": "flowers_" + TIMESTAMP,
    "input_data_config": {
        "dataset_id": dataset_short_id,
    },
    "model_to_upload": {
        "display_name": "flowers_" + TIMESTAMP,
    },
    "training_task_definition": TRAINING_SCHEMA,
    "training_task_inputs": task,
}


print(
    MessageToJson(
        aip.CreateTrainingPipelineRequest(
            parent=PARENT,
            training_pipeline=training_pipeline,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "trainingPipeline": {
    "displayName": "flowers_20210226014942",
    "inputDataConfig": {
      "datasetId": "3094342379910463488"
    },
    "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml",
    "trainingTaskInputs": {
      "model_type": "CLOUD",
      "budget_milli_node_hours": 8000.0,
      "multi_label": false,
      "disable_early_stopping": false
    },
    "modelToUpload": {
      "displayName": "flowers_20210226014942"
    }
  }
}
```


#### Call


In [ ]:
request = clients["pipeline"].create_training_pipeline(
    parent=PARENT,
    training_pipeline=training_pipeline,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/1112934465727889408",
  "displayName": "flowers_20210226014942",
  "inputDataConfig": {
    "datasetId": "3094342379910463488"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml",
  "trainingTaskInputs": {
    "budgetMilliNodeHours": "8000",
    "modelType": "CLOUD"
  },
  "modelToUpload": {
    "displayName": "flowers_20210226014942"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T02:11:57.377842Z",
  "updateTime": "2021-02-26T02:11:57.377842Z"
}
```


In [ ]:
# The full unique ID for the training pipeline
training_pipeline_id = request.name
# The short numeric ID for the training pipeline
training_pipeline_short_id = training_pipeline_id.split("/")[-1]

print(training_pipeline_id)

### [projects.locations.trainingPipelines.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/get)


#### Call


In [ ]:
request = clients["pipeline"].get_training_pipeline(
    name=training_pipeline_id,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/1112934465727889408",
  "displayName": "flowers_20210226014942",
  "inputDataConfig": {
    "datasetId": "3094342379910463488"
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml",
  "trainingTaskInputs": {
    "budgetMilliNodeHours": "8000",
    "modelType": "CLOUD"
  },
  "modelToUpload": {
    "displayName": "flowers_20210226014942"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T02:11:57.377842Z",
  "updateTime": "2021-02-26T02:11:57.377842Z"
}
```


In [ ]:
while True:
    response = clients["pipeline"].get_training_pipeline(name=training_pipeline_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        model_id = response.model_to_upload.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print(model_id)

## Evaluate the model


### [projects.locations.models.evaluations.list](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)


#### Call


In [ ]:
request = clients["model"].list_model_evaluations(
    parent=model_id,
)

#### Response


In [ ]:
import json

model_evaluations = [json.loads(MessageToJson(me.__dict__["_pb"])) for me in request]
# The evaluation slice
evaluation_slice = request.model_evaluations[0].name

print(json.dumps(model_evaluations, indent=2))

*Example output*
```
[
  {
    "name": "projects/116273516712/locations/us-central1/models/6656478578927992832/evaluations/8656839874550169600",
    "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
    "metrics": {
      "confidenceMetrics": [
        {
          "precision": 0.2,
          "recall": 1.0
        },
        {
          "confidenceThreshold": 0.05,
          "recall": 0.98092645,
          "precision": 0.8910891
        },
        {
          "recall": 0.97275203,
          "confidenceThreshold": 0.1,
          "precision": 0.92248064
        },
        {
          "recall": 0.97002727,
          "confidenceThreshold": 0.15,
          "precision": 0.9295039
        },
        {
          "precision": 0.93421054,
          "confidenceThreshold": 0.2,
          "recall": 0.96730244
        },
        {
          "precision": 0.9465241,
          "recall": 0.9645777,
          "confidenceThreshold": 0.25
        },
        {
          "recall": 0.9645777,
          "precision": 0.9516129,
          "confidenceThreshold": 0.3
        },
        {
          "precision": 0.9567568,
          "recall": 0.9645777,
          "confidenceThreshold": 0.35
        },
        {
          "precision": 0.9592391,
          "recall": 0.96185285,
          "confidenceThreshold": 0.4
        },
        {
          "confidenceThreshold": 0.45,
          "precision": 0.96185285,
          "recall": 0.96185285
        },
        {
          "precision": 0.96185285,
          "recall": 0.96185285,
          "confidenceThreshold": 0.5
        },
        {
          "recall": 0.96185285,
          "confidenceThreshold": 0.55,
          "precision": 0.9644809
        },
        {
          "recall": 0.95640326,
          "confidenceThreshold": 0.6,
          "precision": 0.96428573
        },
        {
          "precision": 0.96694213,
          "confidenceThreshold": 0.65,
          "recall": 0.95640326
        },
        {
          "recall": 0.9536785,
          "confidenceThreshold": 0.7,
          "precision": 0.9695291
        },
        {
          "confidenceThreshold": 0.75,
          "precision": 0.9719888,
          "recall": 0.94550407
        },
        {
          "precision": 0.97720796,
          "confidenceThreshold": 0.8,
          "recall": 0.9346049
        },
        {
          "confidenceThreshold": 0.85,
          "recall": 0.9318801,
          "precision": 0.9771429
        },
        {
          "confidenceThreshold": 0.875,
          "recall": 0.9291553,
          "precision": 0.97988504
        },
        {
          "confidenceThreshold": 0.9,
          "precision": 0.98255813,
          "recall": 0.92098093
        },
        {
          "confidenceThreshold": 0.91,
          "precision": 0.9825073,
          "recall": 0.9182561
        },
        {
          "confidenceThreshold": 0.92,
          "recall": 0.91553134,
          "precision": 0.9882353
        },
        {
          "confidenceThreshold": 0.93,
          "recall": 0.9128065,
          "precision": 0.9882006
        },
        {
          "precision": 0.98813057,
          "confidenceThreshold": 0.94,
          "recall": 0.907357
        },
        {
          "precision": 0.990991,
          "recall": 0.89918256,
          "confidenceThreshold": 0.95
        },
        {
          "recall": 0.8855586,
          "precision": 0.9938838,
          "confidenceThreshold": 0.96
        },
        {
          "precision": 0.99380803,
          "recall": 0.8746594,
          "confidenceThreshold": 0.97
        },
        {
          "recall": 0.8692098,
          "precision": 0.99376947,
          "confidenceThreshold": 0.98
        },
        {
          "confidenceThreshold": 0.99,
          "precision": 0.9968254,
          "recall": 0.8555858
        },
        {
          "confidenceThreshold": 0.995,
          "recall": 0.8310627,
          "precision": 1.0
        },
        {
          "recall": 0.8256131,
          "precision": 1.0,
          "confidenceThreshold": 0.996
        },
        {
          "recall": 0.8092643,
          "confidenceThreshold": 0.997,
          "precision": 1.0
        },
        {
          "confidenceThreshold": 0.998,
          "precision": 1.0,
          "recall": 0.79019076
        },
        {
          "precision": 1.0,
          "recall": 0.76021796,
          "confidenceThreshold": 0.999
        },
        {
          "precision": 1.0,
          "confidenceThreshold": 1.0,
          "recall": 0.22888283
        }
      ],
      "confusionMatrix": {
        "rows": [
          [
            80.0,
            0.0,
            0.0,
            0.0,
            0.0
          ],
          [
            3.0,
            85.0,
            0.0,
            2.0,
            0.0
          ],
          [
            0.0,
            1.0,
            67.0,
            1.0,
            1.0
          ],
          [
            1.0,
            1.0,
            1.0,
            60.0,
            0.0
          ],
          [
            3.0,
            0.0,
            0.0,
            0.0,
            61.0
          ]
        ],
        "annotationSpecs": [
          {
            "displayName": "tulips",
            "id": "521556639170428928"
          },
          {
            "displayName": "dandelion",
            "id": "1674478143777275904"
          },
          {
            "displayName": "sunflowers",
            "id": "2827399648384122880"
          },
          {
            "displayName": "daisy",
            "id": "5133242657597816832"
          },
          {
            "id": "7439085666811510784",
            "displayName": "roses"
          }
        ]
      },
      "logLoss": 0.04900711,
      "auPrc": 0.99361706
    },
    "createTime": "2021-02-26T02:36:30.247855Z",
    "sliceDimensions": [
      "annotationSpec"
    ]
  }
]
```


### [projects.locations.models.evaluations.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models.evaluations/get)


#### Call


In [ ]:
request = clients["model"].get_model_evaluation(
    name=evaluation_slice,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/6656478578927992832/evaluations/8656839874550169600",
  "metricsSchemaUri": "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml",
  "metrics": {
    "logLoss": 0.04900711,
    "confusionMatrix": {
      "annotationSpecs": [
        {
          "displayName": "tulips",
          "id": "521556639170428928"
        },
        {
          "displayName": "dandelion",
          "id": "1674478143777275904"
        },
        {
          "displayName": "sunflowers",
          "id": "2827399648384122880"
        },
        {
          "displayName": "daisy",
          "id": "5133242657597816832"
        },
        {
          "id": "7439085666811510784",
          "displayName": "roses"
        }
      ],
      "rows": [
        [
          80.0,
          0.0,
          0.0,
          0.0,
          0.0
        ],
        [
          3.0,
          85.0,
          0.0,
          2.0,
          0.0
        ],
        [
          0.0,
          1.0,
          67.0,
          1.0,
          1.0
        ],
        [
          1.0,
          1.0,
          1.0,
          60.0,
          0.0
        ],
        [
          3.0,
          0.0,
          0.0,
          0.0,
          61.0
        ]
      ]
    },
    "auPrc": 0.99361706,
    "confidenceMetrics": [
      {
        "recall": 1.0,
        "precision": 0.2
      },
      {
        "precision": 0.8910891,
        "confidenceThreshold": 0.05,
        "recall": 0.98092645
      },
      {
        "recall": 0.97275203,
        "precision": 0.92248064,
        "confidenceThreshold": 0.1
      },
      {
        "confidenceThreshold": 0.15,
        "precision": 0.9295039,
        "recall": 0.97002727
      },
      {
        "confidenceThreshold": 0.2,
        "precision": 0.93421054,
        "recall": 0.96730244
      },
      {
        "confidenceThreshold": 0.25,
        "recall": 0.9645777,
        "precision": 0.9465241
      },
      {
        "precision": 0.9516129,
        "recall": 0.9645777,
        "confidenceThreshold": 0.3
      },
      {
        "confidenceThreshold": 0.35,
        "precision": 0.9567568,
        "recall": 0.9645777
      },
      {
        "precision": 0.9592391,
        "recall": 0.96185285,
        "confidenceThreshold": 0.4
      },
      {
        "recall": 0.96185285,
        "precision": 0.96185285,
        "confidenceThreshold": 0.45
      },
      {
        "precision": 0.96185285,
        "recall": 0.96185285,
        "confidenceThreshold": 0.5
      },
      {
        "precision": 0.9644809,
        "recall": 0.96185285,
        "confidenceThreshold": 0.55
      },
      {
        "confidenceThreshold": 0.6,
        "recall": 0.95640326,
        "precision": 0.96428573
      },
      {
        "recall": 0.95640326,
        "precision": 0.96694213,
        "confidenceThreshold": 0.65
      },
      {
        "confidenceThreshold": 0.7,
        "precision": 0.9695291,
        "recall": 0.9536785
      },
      {
        "recall": 0.94550407,
        "confidenceThreshold": 0.75,
        "precision": 0.9719888
      },
      {
        "recall": 0.9346049,
        "precision": 0.97720796,
        "confidenceThreshold": 0.8
      },
      {
        "precision": 0.9771429,
        "confidenceThreshold": 0.85,
        "recall": 0.9318801
      },
      {
        "precision": 0.97988504,
        "confidenceThreshold": 0.875,
        "recall": 0.9291553
      },
      {
        "recall": 0.92098093,
        "confidenceThreshold": 0.9,
        "precision": 0.98255813
      },
      {
        "recall": 0.9182561,
        "confidenceThreshold": 0.91,
        "precision": 0.9825073
      },
      {
        "precision": 0.9882353,
        "confidenceThreshold": 0.92,
        "recall": 0.91553134
      },
      {
        "precision": 0.9882006,
        "confidenceThreshold": 0.93,
        "recall": 0.9128065
      },
      {
        "precision": 0.98813057,
        "recall": 0.907357,
        "confidenceThreshold": 0.94
      },
      {
        "precision": 0.990991,
        "confidenceThreshold": 0.95,
        "recall": 0.89918256
      },
      {
        "precision": 0.9938838,
        "confidenceThreshold": 0.96,
        "recall": 0.8855586
      },
      {
        "recall": 0.8746594,
        "precision": 0.99380803,
        "confidenceThreshold": 0.97
      },
      {
        "confidenceThreshold": 0.98,
        "precision": 0.99376947,
        "recall": 0.8692098
      },
      {
        "precision": 0.9968254,
        "confidenceThreshold": 0.99,
        "recall": 0.8555858
      },
      {
        "confidenceThreshold": 0.995,
        "precision": 1.0,
        "recall": 0.8310627
      },
      {
        "precision": 1.0,
        "recall": 0.8256131,
        "confidenceThreshold": 0.996
      },
      {
        "confidenceThreshold": 0.997,
        "precision": 1.0,
        "recall": 0.8092643
      },
      {
        "recall": 0.79019076,
        "precision": 1.0,
        "confidenceThreshold": 0.998
      },
      {
        "confidenceThreshold": 0.999,
        "recall": 0.76021796,
        "precision": 1.0
      },
      {
        "confidenceThreshold": 1.0,
        "precision": 1.0,
        "recall": 0.22888283
      }
    ]
  },
  "createTime": "2021-02-26T02:36:30.247855Z",
  "sliceDimensions": [
    "annotationSpec"
  ]
}
```


## Make batch predictions


### Make a batch prediction file


In [ ]:
test_items = !gsutil cat $IMPORT_FILE | head -n2

if len(str(test_items[0]).split(",")) == 3:
    _, test_item_1, test_label_1 = str(test_items[0]).split(",")
    _, test_item_2, test_label_2 = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1 = str(test_items[0]).split(",")
    test_item_2, test_label_2 = str(test_items[1]).split(",")

print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

*Example output*:
```
gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg daisy
gs://cloud-ml-data/img/flower_photos/daisy/10140303196_b88d3d6cec.jpg daisy
```


In [ ]:
file_1 = test_item_1.split("/")[-1]
file_2 = test_item_2.split("/")[-1]

! gsutil cp $test_item_1 gs://$BUCKET_NAME/$file_1
! gsutil cp $test_item_2 gs://$BUCKET_NAME/$file_2

test_item_1 = "gs://" + BUCKET_NAME + "/" + file_1
test_item_2 = "gs://" + BUCKET_NAME + "/" + file_2

### Make the batch input file

Let's now make a batch input file, which you will store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each data item (instance). The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the image.
- `mime_type`: The content type. In our example, it is an `jpeg` file.


In [ ]:
import json

import tensorflow as tf

gcs_input_uri = "gs://" + BUCKET_NAME + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {"content": test_item_1, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")
    data = {"content": test_item_2, "mime_type": "image/jpeg"}
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
!gsutil cat $gcs_input_uri

*Example output*:
```
gs://migration-ucaip-trainingaip-20210226014942/test.jsonl
{"content": "gs://migration-ucaip-trainingaip-20210226014942/100080576_f52e8ee070_n.jpg", "mime_type": "image/jpeg"}
{"content": "gs://migration-ucaip-trainingaip-20210226014942/10140303196_b88d3d6cec.jpg", "mime_type": "image/jpeg"}
```


### [projects.locations.batchPredictionJobs.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/create)


#### Request


In [ ]:
parameters = {"confidenceThreshold": 0.5, "maxPredictions": 2}

batch_prediction_job = {
    "display_name": "flowers_" + TIMESTAMP,
    "model": model_id,
    "input_config": {
        "instances_format": "jsonl",
        "gcs_source": {
            "uris": [gcs_input_uri],
        },
    },
    "model_parameters": json_format.ParseDict(parameters, Value()),
    "output_config": {
        "predictions_format": "jsonl",
        "gcs_destination": {
            "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/batch_output/",
        },
    },
    "dedicated_resources": {
        "machine_spec": {
            "machine_type": "n1-standard-2",
            "accelerator_type": 0,
        },
        "starting_replica_count": 1,
        "max_replica_count": 1,
    },
}

print(
    MessageToJson(
        aip.CreateBatchPredictionJobRequest(
            parent=PARENT,
            batch_prediction_job=batch_prediction_job,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "batchPredictionJob": {
    "displayName": "flowers_20210226014942",
    "model": "projects/116273516712/locations/us-central1/models/6656478578927992832",
    "inputConfig": {
      "instancesFormat": "jsonl",
      "gcsSource": {
        "uris": [
          "gs://migration-ucaip-trainingaip-20210226014942/test.jsonl"
        ]
      }
    },
    "modelParameters": {
      "confidenceThreshold": 0.5,
      "maxPredictions": 2.0
    },
    "outputConfig": {
      "predictionsFormat": "jsonl",
      "gcsDestination": {
        "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226014942/batch_output/"
      }
    },
    "dedicatedResources": {
      "machineSpec": {
        "machineType": "n1-standard-2"
      },
      "startingReplicaCount": 1,
      "maxReplicaCount": 1
    }
  }
}
```


#### Call


In [ ]:
request = clients["job"].create_batch_prediction_job(
    parent=PARENT,
    batch_prediction_job=batch_prediction_job,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/7156765165659095040",
  "displayName": "flowers_20210226014942",
  "model": "projects/116273516712/locations/us-central1/models/6656478578927992832",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210226014942/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "maxPredictions": 2.0,
    "confidenceThreshold": 0.5
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226014942/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-02-26T02:36:52.483588Z",
  "updateTime": "2021-02-26T02:36:52.483588Z"
}
```


In [ ]:
# The fully qualified ID for the batch job
batch_job_id = request.name
# The short numeric ID for the batch job
batch_job_short_id = batch_job_id.split("/")[-1]

print(batch_job_id)

### [projects.locations.batchPredictionJobs.get](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.batchPredictionJobs/get)


#### Call


In [ ]:
request = clients["job"].get_batch_prediction_job(
    name=batch_job_id,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/batchPredictionJobs/7156765165659095040",
  "displayName": "flowers_20210226014942",
  "model": "projects/116273516712/locations/us-central1/models/6656478578927992832",
  "inputConfig": {
    "instancesFormat": "jsonl",
    "gcsSource": {
      "uris": [
        "gs://migration-ucaip-trainingaip-20210226014942/test.jsonl"
      ]
    }
  },
  "modelParameters": {
    "confidenceThreshold": 0.5,
    "maxPredictions": 2.0
  },
  "outputConfig": {
    "predictionsFormat": "jsonl",
    "gcsDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226014942/batch_output/"
    }
  },
  "state": "JOB_STATE_PENDING",
  "completionStats": {
    "incompleteCount": "-1"
  },
  "createTime": "2021-02-26T02:36:52.483588Z",
  "updateTime": "2021-02-26T02:36:52.483588Z"
}
```


In [ ]:
def get_latest_predictions(gcs_out_dir):
    """ Get the latest prediction subfolder using the timestamp in the subfolder name"""
    folders = !gsutil ls $gcs_out_dir
    latest = ""
    for folder in folders:
        subfolder = folder.split("/")[-2]
        if subfolder.startswith("prediction-"):
            if subfolder > latest:
                latest = folder[:-1]
    return latest


while True:
    response = clients["job"].get_batch_prediction_job(name=batch_job_id)
    if response.state != aip.JobState.JOB_STATE_SUCCEEDED:
        print("The job has not completed:", response.state)
        if response.state == aip.JobState.JOB_STATE_FAILED:
            break
    else:
        folder = get_latest_predictions(
            response.output_config.gcs_destination.output_uri_prefix
        )
        ! gsutil ls $folder/prediction*.jsonl

        ! gsutil cat $folder/prediction*.jsonl
        break
    time.sleep(60)

*Example output*:
```
gs://migration-ucaip-trainingaip-20210226014942/batch_output/prediction-flowers_20210226014942-2021-02-26T02:36:52.355258Z/predictions_00001.jsonl
{"instance":{"content":"gs://migration-ucaip-trainingaip-20210226014942/10140303196_b88d3d6cec.jpg","mimeType":"image/jpeg"},"prediction":{"ids":["5133242657597816832"],"displayNames":["daisy"],"confidences":[0.9999988]}}
{"instance":{"content":"gs://migration-ucaip-trainingaip-20210226014942/100080576_f52e8ee070_n.jpg","mimeType":"image/jpeg"},"prediction":{"ids":["5133242657597816832"],"displayNames":["daisy"],"confidences":[0.99999106]}}
```


## Make online predictions


### [projects.locations.endpoints.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/create)


#### Request


In [ ]:
endpoint = {"display_name": "flowers_" + TIMESTAMP}

print(
    MessageToJson(
        aip.CreateEndpointRequest(
            parent=PARENT,
            endpoint=endpoint,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "endpoint": {
    "displayName": "flowers_20210226014942"
  }
}
```


#### Call


In [ ]:
request = clients["endpoint"].create_endpoint(
    parent=PARENT,
    endpoint=endpoint,
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/endpoints/3440574193450614784"
}
```


In [ ]:
# The full unique ID for the endpoint
endpoint_id = result.name
# The short numeric ID for the endpoint
endpoint_short_id = endpoint_id.split("/")[-1]

print(endpoint_id)

### [projects.locations.endpoints.deployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/deployModel)


#### Request


In [ ]:
deployed_model = {
    "model": model_id,
    "display_name": "flowers_" + TIMESTAMP,
    "automatic_resources": {"min_replica_count": 1, "max_replica_count": 1},
}

print(
    MessageToJson(
        aip.DeployModelRequest(
            endpoint=endpoint_id,
            deployed_model=deployed_model,
            traffic_split={
                "0": 100,
            },
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/3440574193450614784",
  "deployedModel": {
    "model": "projects/116273516712/locations/us-central1/models/6656478578927992832",
    "displayName": "flowers_20210226014942",
    "automaticResources": {
      "minReplicaCount": 1,
      "maxReplicaCount": 1
    }
  },
  "trafficSplit": {
    "0": 100
  }
}
```


#### Call


In [ ]:
request = clients["endpoint"].deploy_model(
    endpoint=endpoint_id,
    deployed_model=deployed_model,
    traffic_split={
        "0": 100,
    },
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{
  "deployedModel": {
    "id": "5165312113245159424"
  }
}
```


In [ ]:
# The unique ID for the deployed model
deployed_model_id = result.deployed_model.id

print(deployed_model_id)

### [projects.locations.endpoints.predict](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/predict)


#### Prepare file for online prediction

In [ ]:
import base64

import tensorflow as tf

test_item = !gsutil cat $IMPORT_FILE | head -n1

if len(str(test_item[0]).split(",")) == 3:
    _, test_item, test_label = str(test_item[0]).split(",")
else:
    test_item, test_label = str(test_item[0]).split(",")

print(test_item, test_label)

with tf.io.gfile.GFile(test_item, "rb") as f:
    content = f.read()

*Example output*:
```
gs://cloud-ml-data/img/flower_photos/daisy/100080576_f52e8ee070_n.jpg daisy
```


#### Request


In [ ]:
parameters_dict = {
    "confidenceThreshold": 0.5,
    "maxPredictions": 2,
}
parameters = json_format.ParseDict(parameters_dict, Value())

# The format of each instance should conform to the deployed model's prediction input schema.
instances_list = [{"content": base64.b64encode(content).decode("utf-8")}]
instances = [json_format.ParseDict(s, Value()) for s in instances_list]

request = aip.PredictRequest(
    endpoint=endpoint_id,
    parameters=parameters,
)
request.instances.append(instances)

print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "endpoint": "projects/116273516712/locations/us-central1/endpoints/3440574193450614784",
  "instances": [
    [
      {
        "content": "..."
      }
    ]
  ],
  "parameters": {
    "maxPredictions": 2.0,
    "confidenceThreshold": 0.5
  }
}
```


#### Call


In [ ]:
request = clients["prediction"].predict(
    endpoint=endpoint_id,
    instances=instances,
    parameters=parameters,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "predictions": [
    {
      "confidences": [
        0.999991059
      ],
      "ids": [
        "5133242657597816832"
      ],
      "displayNames": [
        "daisy"
      ]
    }
  ],
  "deployedModelId": "5165312113245159424"
}
```


### [projects.locations.endpoints.undeployModel](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.endpoints/undeployModel)


#### Call


In [ ]:
request = clients["endpoint"].undeploy_model(
    endpoint=endpoint_id,
    deployed_model_id=deployed_model_id,
    traffic_split={},
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


## Train and export an Edge model


### [projects.locations.trainingPipelines.create](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.trainingPipelines/create)


#### Request


In [ ]:
task = Value(
    struct_value=Struct(
        fields={
            "disable_early_stopping": Value(bool_value=False),
            "model_type": Value(string_value="MOBILE_TF_VERSATILE_1"),
            "multi_label": Value(bool_value=False),
            "budget_milli_node_hours": Value(number_value=8000),
        }
    )
)

training_pipeline = {
    "display_name": "flowers_edge_" + TIMESTAMP,
    "input_data_config": {
        "dataset_id": dataset_short_id,
        "fraction_split": {
            "training_fraction": 0.8,
            "validation_fraction": 0.1,
            "test_fraction": 0.1,
        },
    },
    "model_to_upload": {
        "display_name": "flowers_edge_" + TIMESTAMP,
    },
    "training_task_definition": TRAINING_SCHEMA,
    "training_task_inputs": task,
}

print(
    MessageToJson(
        aip.CreateTrainingPipelineRequest(
            parent=PARENT,
            training_pipeline=training_pipeline,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "parent": "projects/migration-ucaip-training/locations/us-central1",
  "trainingPipeline": {
    "displayName": "flowers_edge_20210226014942",
    "inputDataConfig": {
      "datasetId": "3094342379910463488",
      "fractionSplit": {
        "trainingFraction": 0.8,
        "validationFraction": 0.1,
        "testFraction": 0.1
      }
    },
    "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml",
    "trainingTaskInputs": {
      "multi_label": false,
      "disable_early_stopping": false,
      "budget_milli_node_hours": 8000.0,
      "model_type": "MOBILE_TF_VERSATILE_1"
    },
    "modelToUpload": {
      "displayName": "flowers_edge_20210226014942"
    }
  }
}
```


#### Call


In [ ]:
request = clients["pipeline"].create_training_pipeline(
    parent=PARENT,
    training_pipeline=training_pipeline,
)

#### Response


In [ ]:
print(MessageToJson(request.__dict__["_pb"]))

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/trainingPipelines/7472017139575029760",
  "displayName": "flowers_edge_20210226014942",
  "inputDataConfig": {
    "datasetId": "3094342379910463488",
    "fractionSplit": {
      "trainingFraction": 0.8,
      "validationFraction": 0.1,
      "testFraction": 0.1
    }
  },
  "trainingTaskDefinition": "gs://google-cloud-aiplatform/schema/trainingjob/definition/automl_image_classification_1.0.0.yaml",
  "trainingTaskInputs": {
    "modelType": "MOBILE_TF_VERSATILE_1",
    "budgetMilliNodeHours": "8000"
  },
  "modelToUpload": {
    "displayName": "flowers_edge_20210226014942"
  },
  "state": "PIPELINE_STATE_PENDING",
  "createTime": "2021-02-26T03:34:33.436419Z",
  "updateTime": "2021-02-26T03:34:33.436419Z"
}
```


In [ ]:
# The full unique ID for the training pipeline
training_pipeline_edge_id = request.name
# The short numeric ID for the training pipeline
training_pipeline_edge_short_id = training_pipeline_edge_id.split("/")[-1]

print(training_pipeline_edge_id)

In [ ]:
while True:
    response = clients["pipeline"].get_training_pipeline(name=training_pipeline_edge_id)
    if response.state != aip.PipelineState.PIPELINE_STATE_SUCCEEDED:
        print("Training job has not completed:", response.state)
        if response.state == aip.PipelineState.PIPELINE_STATE_FAILED:
            break
    else:
        model_edge_id = response.model_to_upload.name
        print("Training Time:", response.end_time - response.start_time)
        break
    time.sleep(60)

print(model_edge_id)

### [projects.locations.models.export](https://cloud.google.com/vertex-ai/docs/reference/rest/v1beta1/projects.locations.models/export)


#### Request


In [ ]:
model_output_config = {
    "export_format_id": "tflite",
    "artifact_destination": {
        "output_uri_prefix": "gs://" + f"{BUCKET_NAME}/export/",
    },
}

print(
    MessageToJson(
        aip.ExportModelRequest(
            name=model_edge_id,
            output_config=model_output_config,
        ).__dict__["_pb"]
    )
)

*Example output*:
```
{
  "name": "projects/116273516712/locations/us-central1/models/1204871229996007424",
  "outputConfig": {
    "exportFormatId": "tflite",
    "artifactDestination": {
      "outputUriPrefix": "gs://migration-ucaip-trainingaip-20210226014942/export/"
    }
  }
}
```


#### Call


In [ ]:
request = clients["model"].export_model(
    name=model_edge_id,
    output_config=model_output_config,
)

#### Response


In [ ]:
result = request.result()

print(MessageToJson(result.__dict__["_pb"]))

*Example output*:
```
{}
```


In [ ]:
model_export_dir = model_output_config["artifact_destination"]["output_uri_prefix"]

! gsutil ls -r $model_export_dir

*Example output*:
```
gs://migration-ucaip-trainingaip-20210226014942/export/:

gs://migration-ucaip-trainingaip-20210226014942/export/model-1204871229996007424/:

gs://migration-ucaip-trainingaip-20210226014942/export/model-1204871229996007424/tflite/:

gs://migration-ucaip-trainingaip-20210226014942/export/model-1204871229996007424/tflite/2021-02-26T04:43:08.209439Z/:
gs://migration-ucaip-trainingaip-20210226014942/export/model-1204871229996007424/tflite/2021-02-26T04:43:08.209439Z/model.tflite
```


# Cleaning up

To clean up all GCP resources used in this project, you can [delete the GCP
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial.


In [ ]:
delete_dataset = True
delete_model = True
delete_endpoint = True
delete_pipeline = True
delete_batchjob = True
delete_edge_model = True
delete_edge_pipeline = True
delete_bucket = True

# Delete the dataset using the Vertex AI fully qualified identifier for the dataset
try:
    if delete_dataset:
        clients["dataset"].delete_dataset(name=dataset_id)
except Exception as e:
    print(e)

# Delete the model using the Vertex AI fully qualified identifier for the model
try:
    if delete_model:
        clients["model"].delete_model(name=model_id)
except Exception as e:
    print(e)

# Delete the endpoint using the Vertex AI fully qualified identifier for the endpoint
try:
    if delete_endpoint:
        clients["endpoint"].delete_endpoint(name=endpoint_id)
except Exception as e:
    print(e)

# Delete the training pipeline using the Vertex AI fully qualified identifier for the training pipeline
try:
    if delete_pipeline:
        clients["pipeline"].delete_training_pipeline(name=training_pipeline_id)
except Exception as e:
    print(e)

# Delete the batch job using the Vertex AI fully qualified identifier for the batch job
try:
    if delete_batchjob:
        clients["job"].delete_batch_prediction_job(name=batch_job_id)
except Exception as e:
    print(e)

# Delete the Edge model using the Vertex AI fully qualified identifier for the Edge model
try:
    if delete_edge_model:
        clients["model"].delete_model(name=model_edge_id)
except Exception as e:
    print(e)

# Delete the Edge training pipeline using the Vertex AI fully qualified identifier for the Edge training pipeline
try:
    if delete_edge_pipeline:
        clients["pipeline"].delete_training_pipeline(name=training_pipeline_edge_id)
except Exception as e:
    print(e)


if delete_bucket and "BUCKET_NAME" in globals():
    ! gsutil rm -r gs://$BUCKET_NAME